In [1]:
import sys;
import os;
import glob;
import math;
import numpy as np;
import glob;
import random;
import time;
import torch;
import torch.optim as optim;
import torch.nn as nn;

sys.path.append(os.getcwd());
sys.path.append(os.path.abspath('../../'));
sys.path.append(os.path.abspath('../../../'));
# sys.path.append(os.path.join(os.getcwd(), 'torch/resources'));
import common.utils as U;
import common.opts as opts;
# import resources.models as models;
import th.resources.calculator as calc;
import common.tlopts as tlopts
# import resources.train_generator as train_generator;
import argparse
from itertools import repeat

In [2]:
import th.resources.models as models

In [3]:
from SharedLibs.datestring import genDataTimeStr, getDateStr

In [4]:
#Reproducibility
seed = 42;
random.seed(seed);
np.random.seed(seed);
torch.manual_seed(seed);
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed);
torch.backends.cudnn.deterministic = True;
torch.backends.cudnn.benchmark = False;

In [5]:
second_pruned_model = "../../trained_models/step_4_second_stage_pruning/pruning_time_2024041710_prunratio85.0/model_second_stage_prun_20240417103352.pt"

## define the variables opt and set values

In [6]:
def getOpts():
    parser = argparse.ArgumentParser(description='Transfer Learning for ACDNet');
    parser.add_argument('--netType', default='ACDNetV2',  required=False);
    parser.add_argument('--data', default='trainSet',  required=False);
    parser.add_argument('--dataset', required=False, default='uec_iot', choices=['2']);
    parser.add_argument('--BC', default=True, action='store_true', help='BC learning');
    parser.add_argument('--strongAugment', default=True,  action='store_true', help='Add scale and gain augmentation');
    #在ipynb中，不能使用parser.parse，要改用parser.parse_known_args()
    opt, unknown = parser.parse_known_args();
    return opt
    # opt = parser.parse_args();

## Define the Generator

In [7]:
class TLGenerator():
    #Generates data for Keras
    def __init__(self, samples=None, labels=None, options=None, classes_dict=None):
        random.seed(42);
        #Initialization
        print(f"length of samples:{len(samples)}")
        self.data = [(samples[i], labels[i]) for i in range (0, len(samples))];
        self.opt = options;
        self.batch_size = options.batchSize;
        self.preprocess_funcs = self.preprocess_setup();
        self.mapdict = classes_dict

    def __len__(self):
        #Denotes the number of batches per epoch
        return int(np.floor(len(self.data) / self.batch_size));
        #return len(self.samples);

    def __getitem__(self, batchIndex):
        #Generate one batch of data
        batchX, batchY = self.generate_batch(batchIndex);
        batchX = np.expand_dims(batchX, axis=1);
        batchX = np.expand_dims(batchX, axis=3);
        return batchX, batchY

    def generate_batch(self, batchIndex):
        #Generates data containing batch_size samples
        sounds = [];
        labels = [];
        indexes = None;
        for i in range(self.batch_size):
            # Training phase of BC learning
            # Select two training examples
            while True:
                sound1, label1 = self.data[random.randint(0, len(self.data) - 1)]
                sound2, label2 = self.data[random.randint(0, len(self.data) - 1)]
                if label1 != label2:
                    break
            sound1 = self.preprocess(sound1)
            sound2 = self.preprocess(sound2)

            # Mix two examples
            r = np.array(random.random())
            sound = U.mix(sound1, sound2, r, self.opt.sr).astype(np.float32)
            # print(f"sound length after U.mix is {len(sound)}")
            # print(f"nClasses:{self.opt.nClasses}, type of mapdict:{type(self.mapdict)}, type of label1:{type(label1)}")
            eye = np.eye(self.opt.nClasses)
            idx1 = self.mapdict[str(label1)]- 1
            idx2 = self.mapdict[str(label2)] - 1
            label = (eye[idx1] * r + eye[idx2] * (1 - r)).astype(np.float32)
            # label = (eye[label1] * r + eye[label2] * (1 - r)).astype(np.float32)

            #For stronger augmentation
            sound = U.random_gain(6)(sound).astype(np.float32)
            # print(f"sound length after U.random_gain is {len(sound)}")
            sounds.append(sound);
            labels.append(label);

        sounds = np.asarray(sounds);
        labels = np.asarray(labels);
        # print(f"batchIndex is {batchIndex}, total sounds is {len(sounds)}")
        # print(f"labels in generate_batch is:\n{labels}")

        return sounds, labels;

    def preprocess_setup(self):
        funcs = []
        if self.opt.strongAugment:
            funcs += [U.random_scale(1.25)]

        funcs += [U.padding(self.opt.inputLength // 2),
                  U.random_crop(self.opt.inputLength),
                  U.normalize(32768.0)]
        return funcs

    def preprocess(self, sound):
        for f in self.preprocess_funcs:
            sound = f(sound)

        return sound;

## Define Trainer

In [19]:
class ReTrainer:
    def __init__(self, opt=None, classes_dict=None):
        self.opt = opt;
        self.testX = None;
        self.testY = None;
        self.bestAcc = 0.0;
        self.bestAccEpoch = 0;
        self.trainGen = getTrainGen(opt,classes_dict=classes_dict)#train_generator.setup(opt, split);
        # self.opt.trainer = self;
        # self.trainGen = getTrainGen(self.opt, self.opt.splits)#train_generator.setup(self.opt, self.opt.split);
        # self.pretrainedmodelpath = "./resources/pretrained_models/acdnet20_20khz_fold4.h5"

    def Train(self):
        train_start_time = time.time();
        state = torch.load(second_pruned_model, map_location="cuda:0")
        weights = state['weight']
        config = state['config']
        print(f"config is {config}")
        net = models.GetACDNetModel(input_len=30225, sr=20000, nclass=self.opt.nClasses, channel_config=config)
        net.load_state_dict(weights);
         #show the acdnet structures
        calc.summary(net,(1,1,30225))
        # net = getPrunedModel(pruned_model_path=pruned_acdnet)
        #print networks parameters' require_grade value
        for k_, v_ in net.named_parameters():
            print(f"{k_}:{v_.requires_grad}")
        print('ACDNet model has been prepared for training');

        calc.summary(net, (1,1,self.opt.inputLength));
        # net = net.cuda();
        # training_text = "Re-Training" if self.opt.retrain else "Training from Scratch";
        # print("{} has been started. You will see update after finishing every training epoch and validation".format(training_text));

        lossFunc = torch.nn.KLDivLoss(reduction='batchmean');
        optimizer = optim.SGD(net.parameters(), lr=self.opt.LR, weight_decay=self.opt.weightDecay, momentum=self.opt.momentum, nesterov=True);

        # self.opt.nEpochs = 1957 if self.opt.split == 4 else 2000;
        for epochIdx in range(self.opt.nEpochs):
            epoch_start_time = time.time();
            optimizer.param_groups[0]['lr'] = self.__get_lr(epochIdx+1);
            cur_lr = optimizer.param_groups[0]['lr'];
            running_loss = 0.0;
            running_acc = 0.0;
            n_batches = math.ceil(len(self.trainGen.data)/self.opt.batchSize);
            for batchIdx in range(n_batches):
                # with torch.no_grad():
                x,y = self.trainGen.__getitem__(batchIdx)
                x = torch.tensor(np.moveaxis(x, 3, 1)).to(self.opt.device);
                y = torch.tensor(y).to(self.opt.device);
                # zero the parameter gradients
                optimizer.zero_grad();

                # forward + backward + optimize
                # outputs = net(x);#in office and use cpu
                x = x.type(torch.FloatTensor) #use apple m2
                outputs = net(x)
                # in office use cpu, need to change to cuda
                # running_acc += (((outputs.data.argmax(dim=1) == y.argmax(dim=1))*1).float().mean()).item();
                # at home use apple m2
                res_y = y.argmax(dim=1)
                res_y = res_y.type(torch.FloatTensor)
                running_acc += ((( outputs.data.argmax(dim=1) == res_y)*1).float().mean()).item();
                y = y.type(torch.FloatTensor)
                
                loss = lossFunc(outputs.log(), y);
                loss.backward();
                optimizer.step();

                running_loss += loss.item();

            tr_acc = (running_acc / n_batches)*100;
            tr_loss = running_loss / n_batches;

            #Epoch wise validation Validation
            epoch_train_time = time.time() - epoch_start_time;

            net.eval();
            val_acc, val_loss = self.__validate(net, lossFunc);
            #Save best model
            self.__save_model(val_acc, epochIdx, net);
            self.__on_epoch_end(epoch_start_time, epoch_train_time, epochIdx, cur_lr, tr_loss, tr_acc, val_loss, val_acc);

            running_loss = 0;
            running_acc = 0;
            net.train();

        total_time_taken = time.time() - train_start_time;
        print("Execution finished in: {}".format(U.to_hms(total_time_taken)));

    def load_test_data(self):
        # data = np.load(os.path.join(self.opt.data, self.opt.dataset, 'test_data_{}khz/fold{}_test4000.npz'.format(self.opt.sr//1000, self.opt.split)), allow_pickle=True);
        data = np.load(self.opt.valData, allow_pickle=True);
        print(f"device is :{self.opt.device}")
        print(f"len of Y:{len(data['y'])}")
        # self.testX = torch.tensor(np.moveaxis(data['x'], 3, 1)).to(self.opt.device);
        dataX = np.moveaxis(data['x'], 3, 1).astype(np.float32);
        self.testX = torch.tensor(dataX).to(self.opt.device);
        self.testY = torch.tensor(data['y']).type(torch.float32).to(self.opt.device);

    def __get_lr(self, epoch):
        divide_epoch = np.array([self.opt.nEpochs * i for i in self.opt.schedule]);
        decay = sum(epoch > divide_epoch);
        if epoch <= self.opt.warmup:
            decay = 1;
        return self.opt.LR * np.power(0.1, decay);

    def __get_batch(self, index):
        x = self.trainX[index*self.opt.batchSize : (index+1)*self.opt.batchSize];
        y = self.trainY[index*self.opt.batchSize : (index+1)*self.opt.batchSize];
        return x.to(self.opt.device), y.to(self.opt.device);

    def __validate(self, net, lossFunc):
        if self.testX is None:
            self.load_test_data();
        net.eval();
        with torch.no_grad():
            y_pred = None;
            batch_size = len(self.testX);#(self.opt.batchSize//self.opt.nCrops)*self.opt.nCrops;
#             for idx in range(math.ceil(len(self.testX)/batch_size)):
#             for idx in range(len(self.testX)):
#             x = self.testX[idx*batch_size : (idx+1)*batch_size];
            x = self.testX[:];
            x = torch.tensor(x)
            x = x.type(torch.FloatTensor) # use apple mp2
            scores = net(x);
            y_pred = scores.data if y_pred is None else torch.cat((y_pred, scores.data));
            acc, loss = self.__compute_accuracy(y_pred, self.testY, lossFunc);
#         with torch.no_grad():
#             y_pred = None;
#             batch_size = (self.opt.batchSize//self.opt.nCrops)*self.opt.nCrops;
#             for idx in range(math.ceil(len(self.testX)/batch_size)):
#                 x = self.testX[idx*batch_size : (idx+1)*batch_size];
#                 scores = net(x);
#                 y_pred = scores.data if y_pred is None else torch.cat((y_pred, scores.data));

#             acc, loss = self.__compute_accuracy(y_pred, self.testY, lossFunc);
        net.train();
        return acc, loss;

    #Calculating average prediction (10 crops) and final accuracy
    def __compute_accuracy(self, y_pred, y_target, lossFunc):
        print(f"shape of y_pred:{y_pred.shape}");
        print(f"shape of y_target:{y_target.shape}");
        
        with torch.no_grad():
            #Reshape to shape theme like each sample comtains 10 samples, calculate mean and find theindices that has highest average value for each sample
            if self.opt.nCrops == 1:
                y_pred = y_pred.argmax(dim=1);
                y_target = y_target.argmax(dim=1);
            else:
                y_pred = (y_pred.reshape(y_pred.shape[0]//self.opt.nCrops, self.opt.nCrops, y_pred.shape[1])).mean(dim=1).argmax(dim=1);
                y_target = (y_target.reshape(y_target.shape[0]//self.opt.nCrops, self.opt.nCrops, y_target.shape[1])).mean(dim=1).argmax(dim=1);
                print(f"after: len of y_pred:{len(y_pred)}, len of y_target:{len(y_target)}")
            y_target = y_target.cpu() #use apple m2, in office use cuda
            acc = (((y_pred==y_target)*1).float().mean()*100).item();
            # valLossFunc = torch.nn.KLDivLoss();
            loss = lossFunc(y_pred.float().log(), y_target.float()).item();
            # loss = 0.0;
        return acc, loss;

    def __on_epoch_end(self, start_time, train_time, epochIdx, lr, tr_loss, tr_acc, val_loss, val_acc):
        epoch_time = time.time() - start_time;
        val_time = epoch_time - train_time;
        line = 'SP-{} Epoch: {}/{} | Time: {} (Train {}  Val {}) | Train: LR {}  Loss {:.2f}  Acc {:.2f}% | Val: Loss {:.2f}  Acc(top1) {:.2f}% | HA {:.2f}@{}\n'.format(
            self.opt.splits, epochIdx+1, self.opt.nEpochs, U.to_hms(epoch_time), U.to_hms(train_time), U.to_hms(val_time),
            lr, tr_loss, tr_acc, val_loss, val_acc, self.bestAcc, self.bestAccEpoch);
        # print(line)
        sys.stdout.write(line);
        sys.stdout.flush();

    def __save_model(self, acc, epochIdx, net):
        if acc > self.bestAcc:
            # dir = os.getcwd();
            # fname = "{}/torch/trained_models/{}_fold{}.pt";
            # old_model = fname.format(dir, self.opt.model_name.lower(), self.opt.split);
            # if os.path.isfile(old_model):
            #     os.remove(old_model);
            self.bestAcc = acc;
            self.bestAccEpoch = epochIdx +1;
            save_model_name = self.opt.model_name.format(self.opt.pruningRatio*100,acc, epochIdx, genDataTimeStr());
            save_model_fullpath = self.opt.saveDir + save_model_name;
            print(f"save model to {save_model_fullpath}")
            torch.save({'weight':net.state_dict(), 'config':net.ch_config}, save_model_fullpath);


In [20]:
def getTrainGen(opt=None, split=None, classes_dict=None):
    dataset = np.load(opt.trainData, allow_pickle=True);
    train_sounds = []
    train_labels = []
    train_sounds = dataset['fold{}'.format(1)].item()['sounds']
    train_labels = dataset['fold{}'.format(1)].item()['labels']
    trainGen = TLGenerator(train_sounds, train_labels, opt, classes_dict=classes_dict);
    trainGen.preprocess_setup();
    return trainGen

In [21]:
""" recording training settings and results
weight-pruning-ratio:
final acc:93.03
epoch:539
opt.batchSize = 32;
opt.LR = 0.05;
opt.weightDecay = 5e-3;
opt.momentum = 0.04;
opt.nEpochs = 1000;
opt.schedule = [0.15, 0.3, 0.45];
opt.warmup = 10;
------------------------------------

------------------------------------

"""

' recording training settings and results\nweight-pruning-ratio:\nfinal acc:\nepoch:\nopt.batchSize = 32;\nopt.LR = 0.05;\nopt.weightDecay = 5e-3;\nopt.momentum = 0.04;\nopt.nEpochs = 1000;\nopt.schedule = [0.15, 0.3, 0.45];\nopt.warmup = 10;\n------------------------------------\n\n------------------------------------\n\n'

In [22]:
"""
epoch:93
acc:
opt.batchSize = 64;
opt.LR = 0.1;
opt.weightDecay = 5e-3;
opt.momentum = 0.09;
opt.nEpochs = 1000;
opt.schedule = [0.03, 0.06, 0.09];
opt.warmup = 10;
"""

'\nepoch:\nacc:\nopt.batchSize = 32;\nopt.LR = 0.01;\nopt.weightDecay = 5e-3;\nopt.momentum = 0.9;\nopt.nEpochs = 1000;\nopt.schedule = [0.03, 0.06, 0.09];\nopt.warmup = 10;\n'

In [23]:
def main():
    map_dict_train = {
        '52':1, #alarm
        '56':2, #moaning
        '99':3, #other_sounds
    };
    
    opt = getOpts();
    opt.pruningRatio = 0.85
    save_dir = "../../trained_models/step_5_retrain_after_step_4/after_second_pruning_time_{}_prunratio{}/".format(getDateStr(),opt.pruningRatio)
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    print(f"save dir is: {save_dir}");
    opt.trainData = "../../../datasets/CurrentUse/generated_datasets/train/single_fold_train_20240416111733.npz"
    opt.valData = "../../../datasets/CurrentUse/generated_datasets/validation/final_single_val_20240416135817.npz"
    #Leqarning settings
    
    opt.batchSize = 64;
    opt.LR = 0.1;
    opt.weightDecay = 5e-3;
    opt.momentum = 0.09;
    opt.nEpochs = 1200;
    opt.schedule = [0.3, 0.6, 0.9];
    opt.warmup = 10;
    
    opt.saveDir = save_dir;
    # opt.device="mps";
    # if torch.backends.mps.is_available():
    #     opt.device="mps"; #for apple m2 gpu
    if torch.cuda.is_available():
        opt.device="cuda:0"; #for nVidia gpu
    else:
        opt.device="cpu"
    print(f"***Use device:{opt.device}");
    # opt.device = torch.device("cuda:0" if  else "cpu");
    #Basic Net Settings
    opt.nClasses = 3
    opt.nFolds = 1;
    opt.splits = [i for i in range(1, opt.nFolds + 1)];
    opt.sr = 20000;
    opt.inputLength = 30225;
    #Test data
    opt.nCrops = 2;
    opt.model_name = "retrained_model_after_second_pruning_ratio{}_acc{}_{}th_epoch_{}.pt"
    #Starting retraining process
    trainer = ReTrainer(opt=opt, classes_dict=map_dict_train);
    trainer.Train();

In [24]:
main()

save dir is: ../../trained_models/step_5_retrain_after_step_4/after_second_pruning_time_2024041812_prunratio0.85/
***Use device:cuda:0
length of samples:727
config is [7, 32, 13, 15, 15, 22, 19, 35, 33, 50, 62, 3]
+----------------------------------------------------------------------------+
+                           Pytorch Model Summary                            +
------------------------------------------------------------------------------
   Layer (type)       Input Shape      Output Shape    Param #      FLOPS #
       Conv2d-1     (1, 1, 30225)     (7, 1, 15109)         63      951,867
  BatchNorm2d-2     (7, 1, 15109)     (7, 1, 15109)         14            0
         ReLu-3     (7, 1, 15109)     (7, 1, 15109)          0      105,763
       Conv2d-4     (7, 1, 15109)     (32, 1, 7553)      1,120    8,459,360
  BatchNorm2d-5     (32, 1, 7553)     (32, 1, 7553)         64            0
         ReLu-6     (32, 1, 7553)     (32, 1, 7553)          0      241,696
    MaxPool2d-7  

/tmp/ipykernel_105703/3459676549.py:123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x)


shape of y_pred:torch.Size([574, 3])
shape of y_target:torch.Size([574, 3])
after: len of y_pred:287, len of y_target:287
save model to ../../trained_models/step_5_retrain_after_step_4/after_second_pruning_time_2024041812_prunratio0.85/retrained_model_after_second_pruning_ratio85.0_acc69.68641662597656_0th_epoch_20240418121444.pt
SP-[1] Epoch: 1/1200 | Time: 0m08s (Train 0m07s  Val 0m01s) | Train: LR 0.010000000000000002  Loss 0.98  Acc 39.06% | Val: Loss -0.10  Acc(top1) 69.69% | HA 69.69@1
shape of y_pred:torch.Size([574, 3])
shape of y_target:torch.Size([574, 3])
after: len of y_pred:287, len of y_target:287
save model to ../../trained_models/step_5_retrain_after_step_4/after_second_pruning_time_2024041812_prunratio0.85/retrained_model_after_second_pruning_ratio85.0_acc74.91289520263672_1th_epoch_20240418121452.pt
SP-[1] Epoch: 2/1200 | Time: 0m07s (Train 0m07s  Val 0m00s) | Train: LR 0.010000000000000002  Loss 0.63  Acc 41.80% | Val: Loss nan  Acc(top1) 74.91% | HA 74.91@2
shape of